In [1]:
import pandas as pd
import numpy as np
import surprise
import apyori
# from surprise.prediction_algorithms.knns import KNNBasic
from surprise import KNNBasic, SVD
from surprise.accuracy import rmse



In [2]:
data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()

In [3]:
testSet = trainingSet.build_testset()

In [4]:
# Найдите количество пользователей в trainingSet.
trainingSet.n_users

943

In [5]:
# Найдите количество items в trainingSet.
trainingSet.n_items


1682

In [6]:
# Выведите среднее значение по всем показателям рейтинга для trainingSet. Ответ округлите до сотых.
np.mean(list(trainingSet.all_ratings()), axis=0)[2].round(2)

3.53

In [7]:
# Проведите тренировку алгоритма user-based коллаборативной фильтрации KNNBasic.
# Постройте рекомендации testset c использованием этого алгоритма.
# Выведите предсказание под \(8\)-ым индексом. В качестве ответа укажите id рекомендованного фильма.
sim_options = {"user_based": True}
model_knn = KNNBasic(sim_options=sim_options)
predictions_knn = model_knn.fit(trainingSet).test(testSet)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [8]:
predictions_knn[8]

Prediction(uid='196', iid='663', r_ui=5.0, est=4.3885580618909685, details={'actual_k': 40, 'was_impossible': False})

In [9]:
# Проведите тренировку алгоритма SVD. Укажите параметр random_state=829.
# Постройте рекомендации testset c использованием этого алгоритма.
# Выведите предсказание под \(42\)-ым индексом.
# В качестве ответа укажите разницу между истинным значением рейтинга для этого предсказания и оцениваемым.
# Округлите до тысячных.
model_svd = SVD(random_state=829)
predictions_svd = model_svd.fit(trainingSet).test(testSet)

In [10]:
round(abs(predictions_svd[42].r_ui - predictions_svd[42].est), 3)


0.424

In [12]:
# Рассчитайте метрику качества RMSE для предсказаний через функцию surprise.accuracy.rmse() 
# для обоих обученных ранее алгоритмов.
# В качестве ответа укажите значение RMSE для того алгоритма, который дал лучший результат.
# Округлите ответ до двух знаков после точки-разделителя.

print(round(rmse(predictions_knn), 2))
print(round(rmse(predictions_svd), 2))

RMSE: 0.7781
0.78
RMSE: 0.6747
0.67


In [13]:
# Cформируем датафрейм из нашего датасета
df = pd.DataFrame(data.raw_ratings)
df.columns = ["user_id", "movie_id", "rating", 3]
df.head()

,user_id,movie_id,rating,3
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [47]:
# Сформируйте для каждого user_id текстовую строку, содержащую набор movie_id, получивших оценку \(4\) и выше.
# Используйте пробел (" ") в качестве разделителя.
# Для проверки результата введите строку для user_id 100.
df_4up = (
    df[df.rating >= 4]
    .groupby("user_id")["movie_id"]
    .apply(lambda movies: " ".join([str(movie) for movie in movies]))
)
# df_4up.reset_index(inplace=True)
# df_4up[df_4up.user_id=="100"]
       
# good = (
#     df[df["Rating"] == 5]
#     .groupby("Cust_Id")["Movie_Id"]
#     .apply(lambda r: " ".join([str(A) for A in r]))
# )


In [36]:
df_4up[df_4up.user_id=="100"].movie_id.to_list()


['344 355 750 302 691 316 752 313 879 300 328 1235 690 347 269 258 900 294 272 751 898 315']

In [48]:
df_4up.head()

user_id
1      61 33 160 20 202 171 265 47 222 253 113 227 90...
10     16 486 611 7 100 488 285 504 289 340 505 489 6...
100    344 355 750 302 691 316 752 313 879 300 328 12...
101    405 121 24 284 50 237 181 924 257 742 255 117 ...
102    195 307 89 202 186 183 98 208 510 50 101 588 1...
Name: movie_id, dtype: object

In [63]:
# Используя алгоритм apriory из модуля apyory, рассчитайте ассоциативные правила для полученного
# набора фильмов с положительными оценками. Будем считать, что хороший рейтинг — это когда оценка >=4.
# Используйте следующие ограничения:
#   минимальный support 0.2,
#   минимальное значение confidence 0.3,
#   минимальное значение lift 2,
#   минимальная длина 2.

association_rules = apyori.apriori(
    df_4up.apply(lambda r: r.split(" ")),
    min_support=0.2,
    min_confidence=0.3,
    min_lift=2,
    min_length=2,
)

asr_df = pd.DataFrame(columns=["from", "to", "confidence", "support", "lift"])
for item in association_rules:
    pair = item[0]
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] = (
        " ".join(list(item[2][0][0])),
        " ".join(list(item[2][0][1])),
        item[2][0][2],
        item[1],
        item[2][0][3],
    )

print(asr_df.shape[0])
print(asr_df["from"].value_counts())

10
174    5
172    4
173    1
Name: from, dtype: int64
